# Predict the winner of a basketball game

# RNN Sequence classifier

In this notebook, we will predict the winner of a basketball game based on the scores observed in the first 3 quarters of the game. Each column represents the beginning of a minute during the game. (There are 12 minutes in each quarter. There are  3 quarters in the data, so we have 36 columns as input variables in chronological order.) The values captured in each column represent the score difference observed at that minute (home score minus away score)<br><br>

The last column `W` represents whether the home team (1) or the away team (0) won the game. This is the target variable. <br><br>
**Our unit of analysis is a single game.**

In [1]:
# Common imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

## Get the data

In [2]:
data = pd.read_csv("basketball.csv")

In [3]:
data.shape

(1230, 37)

In [4]:
data.head()

,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,...,M28,M29,M30,M31,M32,M33,M34,M35,M36,W
0,-2,-1,1,1,-1,-3,-1,0,3,6,...,9,11,10,7,7,4,6,2,1,1
1,0,2,7,6,10,8,8,6,0,6,...,7,11,11,14,14,15,13,13,13,0
2,0,-2,2,0,5,4,5,3,5,3,...,9,11,13,13,12,17,15,15,12,1
3,0,2,0,3,4,3,5,4,3,1,...,10,6,7,7,8,8,8,8,8,1
4,0,-2,-2,0,3,-2,-7,-5,-7,-4,...,10,10,15,13,11,11,11,13,10,1


In [5]:
y = data['W']
x = data.drop('W', axis=1)

## Split the data

In [6]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

## Data Transformation

In [7]:
#Target variables need to be an array with integer type
train_y = np.array(train_y)
test_y = np.array(test_y)

train_y = train_y.astype(np.int32)
test_y = test_y.astype(np.int32)



In [8]:
#Check the first 10 values of the train_y data set
train_y[0:10]

array([1, 1, 1, 1, 0, 1, 0, 1, 0, 1])

In [9]:
#Convert input variables to a 2-D array with float data type
train_x= np.array(train_x)
test_x= np.array(test_x)

train_x = train_x.astype(np.float32)
test_x = test_x.astype(np.float32)

In [10]:
train_x

array([[ 0., -2., -4., ...,  1.,  1.,  2.],
       [-2.,  1.,  2., ..., -8., -3., -1.],
       [-1., -1.,  2., ...,  9.,  9., 10.],
       ...,
       [-2.,  1., -5., ...,  2.,  5.,  7.],
       [ 0.,  4.,  3., ..., -9., -7., -9.],
       [ 0.,  4.,  4., ..., 32., 33., 33.]], dtype=float32)

In [11]:
#Keras expects a different input format:
#Data needs to have 3 dimensions

train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))



In [12]:
train_x.shape, train_y.shape

((861, 36, 1), (861,))

In [13]:
train_x

array([[[ 0.],
        [-2.],
        [-4.],
        ...,
        [ 1.],
        [ 1.],
        [ 2.]],

       [[-2.],
        [ 1.],
        [ 2.],
        ...,
        [-8.],
        [-3.],
        [-1.]],

       [[-1.],
        [-1.],
        [ 2.],
        ...,
        [ 9.],
        [ 9.],
        [10.]],

       ...,

       [[-2.],
        [ 1.],
        [-5.],
        ...,
        [ 2.],
        [ 5.],
        [ 7.]],

       [[ 0.],
        [ 4.],
        [ 3.],
        ...,
        [-9.],
        [-7.],
        [-9.]],

       [[ 0.],
        [ 4.],
        [ 4.],
        ...,
        [32.],
        [33.],
        [33.]]], dtype=float32)

# Baseline Accuracy

In [14]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_y)

DummyClassifier(strategy='most_frequent')

In [15]:
from sklearn.metrics import accuracy_score

In [16]:
#Baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_y, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.5667828106852497


In [17]:
#Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_y, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.5176151761517616


# A normal (cross-sectional) NN

This model assumes that the data is NOT a time-series data set. It treats the data as cross-sectional and the columns being independent of each other.

In [18]:
model = keras.models.Sequential([
    
    keras.layers.Flatten(input_shape=[36, 1]),
    keras.layers.Dense(36, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
    
])

In [19]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_y, epochs=50,
                    validation_data=(test_x, test_y))

Epoch 1/50
27/27 [==============================] - 2s 19ms/step - loss: 0.8566 - accuracy: 0.7375 - val_loss: 0.8532 - val_accuracy: 0.6667
Epoch 2/50
27/27 [==============================] - 0s 6ms/step - loss: 0.4842 - accuracy: 0.7816 - val_loss: 0.5283 - val_accuracy: 0.7669
Epoch 3/50
27/27 [==============================] - 0s 6ms/step - loss: 0.4286 - accuracy: 0.8037 - val_loss: 0.5202 - val_accuracy: 0.7805
Epoch 4/50
27/27 [==============================] - 0s 6ms/step - loss: 0.3970 - accuracy: 0.8258 - val_loss: 0.5600 - val_accuracy: 0.7236
Epoch 5/50
27/27 [==============================] - 0s 6ms/step - loss: 0.3788 - accuracy: 0.8258 - val_loss: 0.5425 - val_accuracy: 0.7453
Epoch 6/50
27/27 [==============================] - 0s 6ms/step - loss: 0.3666 - accuracy: 0.8235 - val_loss: 0.5140 - val_accuracy: 0.7642
Epoch 7/50
27/27 [==============================] - 0s 6ms/step - loss: 0.3487 - accuracy: 0.8525 - val_loss: 0.5564 - val_accuracy: 0.7534
Epoch 8/50
27/27 [=

In [20]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.7438554763793945, 0.7696477174758911]

In [21]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.74
accuracy: 76.96%


# Simple RNN with one layer

In [22]:
n_steps = 36
n_inputs = 1


model = keras.models.Sequential([
    
    keras.layers.SimpleRNN(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [23]:
from tensorflow.keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

In [24]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_y, epochs=50,
                    validation_data=(test_x, test_y), callbacks=callback)

Epoch 1/50
27/27 [==============================] - 4s 36ms/step - loss: 0.4742 - accuracy: 0.7619 - val_loss: 0.4280 - val_accuracy: 0.7940
Epoch 2/50
27/27 [==============================] - 0s 17ms/step - loss: 0.3998 - accuracy: 0.8118 - val_loss: 0.4237 - val_accuracy: 0.7940
Epoch 3/50
27/27 [==============================] - 0s 16ms/step - loss: 0.3910 - accuracy: 0.8281 - val_loss: 0.4427 - val_accuracy: 0.7805
Epoch 4/50
27/27 [==============================] - 0s 17ms/step - loss: 0.3858 - accuracy: 0.8142 - val_loss: 0.4502 - val_accuracy: 0.7615
Epoch 5/50
27/27 [==============================] - 0s 16ms/step - loss: 0.3808 - accuracy: 0.8188 - val_loss: 0.4636 - val_accuracy: 0.7669
Epoch 6/50
27/27 [==============================] - 0s 17ms/step - loss: 0.3732 - accuracy: 0.8223 - val_loss: 0.4360 - val_accuracy: 0.7967
Epoch 7/50
27/27 [==============================] - 0s 17ms/step - loss: 0.3643 - accuracy: 0.8293 - val_loss: 0.4568 - val_accuracy: 0.7696
Epoch 7: earl

In [25]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.45682111382484436, 0.7696477174758911]

In [26]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.46
accuracy: 76.96%


In [27]:
# Predictions are probabilities.

predictions = model.predict(test_x)

12/12 [==============================] - 0s 4ms/step


In [28]:
# Rounding the probabilities determines 1 or 0

np.round(predictions)

array([[0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],

In [29]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_y, np.round(predictions))

array([[128,  50],
       [ 35, 156]], dtype=int64)

# Deep RNN

**Be careful: when stacking RNN layers, you have to set "return_sequences" to True. This enables the layer to send a "sequence" of values to the next layer -- jut like how it uses a sequence of values for training.**

**Since the last layer is DENSE, it can't take sequence data. Therefore, you CANNOT return sequences from the previous layer. So, remove** `return_sequences` **from previous layer.**

In [30]:
n_steps = 36
n_inputs = 1


model = keras.models.Sequential([
    keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[n_steps, n_inputs] ),
    keras.layers.SimpleRNN(32, return_sequences=True),
    keras.layers.SimpleRNN(32), 
    keras.layers.Dense(1, activation='sigmoid')
])


In [31]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
27/27 [==============================] - 9s 78ms/step - loss: 0.4485 - accuracy: 0.7898 - val_loss: 0.4517 - val_accuracy: 0.7940
Epoch 2/20
27/27 [==============================] - 1s 40ms/step - loss: 0.4124 - accuracy: 0.8060 - val_loss: 0.4548 - val_accuracy: 0.7209
Epoch 3/20
27/27 [==============================] - 1s 39ms/step - loss: 0.3937 - accuracy: 0.8153 - val_loss: 0.4690 - val_accuracy: 0.7588
Epoch 4/20
27/27 [==============================] - 1s 40ms/step - loss: 0.3774 - accuracy: 0.8165 - val_loss: 0.4589 - val_accuracy: 0.7615
Epoch 5/20
27/27 [==============================] - 1s 40ms/step - loss: 0.3800 - accuracy: 0.8200 - val_loss: 0.4635 - val_accuracy: 0.7995
Epoch 6/20
27/27 [==============================] - 1s 40ms/step - loss: 0.3902 - accuracy: 0.8211 - val_loss: 0.4438 - val_accuracy: 0.7778
Epoch 7/20
27/27 [==============================] - 1s 39ms/step - loss: 0.3642 - accuracy: 0.8269 - val_loss: 0.4551 - val_accuracy: 0.7778
Epoch 8/20
27

In [32]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.44745948910713196, 0.7804877758026123]

In [33]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.45
accuracy: 78.05%


# LSTM with one layer

In [34]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    
    keras.layers.LSTM(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [35]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
27/27 [==============================] - 7s 67ms/step - loss: 0.4751 - accuracy: 0.7793 - val_loss: 0.4393 - val_accuracy: 0.7913
Epoch 2/20
27/27 [==============================] - 1s 29ms/step - loss: 0.4100 - accuracy: 0.8200 - val_loss: 0.4282 - val_accuracy: 0.7967
Epoch 3/20
27/27 [==============================] - 1s 28ms/step - loss: 0.4008 - accuracy: 0.8118 - val_loss: 0.4298 - val_accuracy: 0.7913
Epoch 4/20
27/27 [==============================] - 1s 30ms/step - loss: 0.3996 - accuracy: 0.8223 - val_loss: 0.4214 - val_accuracy: 0.7967
Epoch 5/20
27/27 [==============================] - 1s 29ms/step - loss: 0.3983 - accuracy: 0.8188 - val_loss: 0.4313 - val_accuracy: 0.7751
Epoch 6/20
27/27 [==============================] - 1s 29ms/step - loss: 0.3929 - accuracy: 0.8130 - val_loss: 0.4226 - val_accuracy: 0.8049
Epoch 7/20
27/27 [==============================] - 1s 29ms/step - loss: 0.3892 - accuracy: 0.8095 - val_loss: 0.4428 - val_accuracy: 0.7832
Epoch 8/20
27

In [36]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.43633052706718445, 0.7750677466392517]

In [37]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.44
accuracy: 77.51%


# LSTM with more layers

In [38]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(32, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation='sigmoid')
])

In [39]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
27/27 [==============================] - 17s 171ms/step - loss: 0.5180 - accuracy: 0.7433 - val_loss: 0.4581 - val_accuracy: 0.7913
Epoch 2/20
27/27 [==============================] - 2s 78ms/step - loss: 0.4207 - accuracy: 0.7979 - val_loss: 0.4251 - val_accuracy: 0.7669
Epoch 3/20
27/27 [==============================] - 2s 81ms/step - loss: 0.4094 - accuracy: 0.8130 - val_loss: 0.4308 - val_accuracy: 0.7669
Epoch 4/20
27/27 [==============================] - 2s 80ms/step - loss: 0.3994 - accuracy: 0.8107 - val_loss: 0.4482 - val_accuracy: 0.7561
Epoch 5/20
27/27 [==============================] - 2s 78ms/step - loss: 0.4112 - accuracy: 0.8177 - val_loss: 0.4367 - val_accuracy: 0.7696
Epoch 6/20
27/27 [==============================] - 2s 79ms/step - loss: 0.4005 - accuracy: 0.8084 - val_loss: 0.4271 - val_accuracy: 0.7859
Epoch 7/20
27/27 [==============================] - 2s 79ms/step - loss: 0.3900 - accuracy: 0.8223 - val_loss: 0.4430 - val_accuracy: 0.7886
Epoch 7: ea

In [40]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.4430147111415863, 0.7886179089546204]

In [41]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.44
accuracy: 78.86%


# GRU with one layer

In [42]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [43]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
27/27 [==============================] - 8s 74ms/step - loss: 0.4657 - accuracy: 0.7607 - val_loss: 0.4342 - val_accuracy: 0.7859
Epoch 2/20
27/27 [==============================] - 1s 32ms/step - loss: 0.4024 - accuracy: 0.8188 - val_loss: 0.4285 - val_accuracy: 0.7913
Epoch 3/20
27/27 [==============================] - 1s 31ms/step - loss: 0.3974 - accuracy: 0.8153 - val_loss: 0.4268 - val_accuracy: 0.7778
Epoch 4/20
27/27 [==============================] - 1s 32ms/step - loss: 0.3960 - accuracy: 0.8118 - val_loss: 0.4287 - val_accuracy: 0.7778
Epoch 5/20
27/27 [==============================] - 1s 31ms/step - loss: 0.4000 - accuracy: 0.8177 - val_loss: 0.4265 - val_accuracy: 0.7913
Epoch 6/20
27/27 [==============================] - 1s 31ms/step - loss: 0.3932 - accuracy: 0.8153 - val_loss: 0.4269 - val_accuracy: 0.7967
Epoch 7/20
27/27 [==============================] - 1s 31ms/step - loss: 0.3853 - accuracy: 0.8130 - val_loss: 0.4340 - val_accuracy: 0.7859
Epoch 8/20
27

In [44]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.43816283345222473, 0.7669376730918884]

In [45]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.44
accuracy: 76.69%


# GRU with more layers

In [46]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(32, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(32, return_sequences=True),
    keras.layers.GRU(32),
    keras.layers.Dense(1, activation='sigmoid')
])

In [47]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
27/27 [==============================] - 17s 165ms/step - loss: 0.4522 - accuracy: 0.7875 - val_loss: 0.4317 - val_accuracy: 0.7913
Epoch 2/20
27/27 [==============================] - 2s 81ms/step - loss: 0.4090 - accuracy: 0.8211 - val_loss: 0.4239 - val_accuracy: 0.7724
Epoch 3/20
27/27 [==============================] - 2s 84ms/step - loss: 0.4028 - accuracy: 0.8095 - val_loss: 0.4334 - val_accuracy: 0.7751
Epoch 4/20
27/27 [==============================] - 2s 85ms/step - loss: 0.3968 - accuracy: 0.8200 - val_loss: 0.4443 - val_accuracy: 0.7778
Epoch 5/20
27/27 [==============================] - 2s 85ms/step - loss: 0.4039 - accuracy: 0.8142 - val_loss: 0.4401 - val_accuracy: 0.7778
Epoch 6/20
27/27 [==============================] - 2s 83ms/step - loss: 0.4113 - accuracy: 0.8084 - val_loss: 0.4244 - val_accuracy: 0.8049
Epoch 7/20
27/27 [==============================] - 2s 84ms/step - loss: 0.3944 - accuracy: 0.8049 - val_loss: 0.4341 - val_accuracy: 0.7886
Epoch 7: ea

In [48]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.43410399556159973, 0.7886179089546204]

In [49]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.43
accuracy: 78.86%


# Conv1D

In [50]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.Conv1D(filters=10, kernel_size=3, strides=2, padding="valid", input_shape=[n_steps, n_inputs]),
    keras.layers.Conv1D(filters=20, kernel_size=3, strides=1, padding="valid", dilation_rate=2),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 17, 10)            40        
                                                                 
 conv1d_1 (Conv1D)           (None, 13, 20)            620       
                                                                 
 lstm_4 (LSTM)               (None, 13, 32)            6784      
                                                                 
 lstm_5 (LSTM)               (None, 32)                8320      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 15,797
Trainable params: 15,797
Non-trainable params: 0
_________________________________________________________________


In [51]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
27/27 [==============================] - 13s 97ms/step - loss: 0.5301 - accuracy: 0.7387 - val_loss: 0.4732 - val_accuracy: 0.7832
Epoch 2/20
27/27 [==============================] - 1s 29ms/step - loss: 0.4361 - accuracy: 0.7944 - val_loss: 0.4651 - val_accuracy: 0.7588
Epoch 3/20
27/27 [==============================] - 1s 28ms/step - loss: 0.4173 - accuracy: 0.7909 - val_loss: 0.4533 - val_accuracy: 0.7886
Epoch 4/20
27/27 [==============================] - 1s 30ms/step - loss: 0.4017 - accuracy: 0.8084 - val_loss: 0.4817 - val_accuracy: 0.7480
Epoch 5/20
27/27 [==============================] - 1s 29ms/step - loss: 0.4091 - accuracy: 0.8107 - val_loss: 0.4618 - val_accuracy: 0.7669
Epoch 6/20
27/27 [==============================] - 1s 28ms/step - loss: 0.3826 - accuracy: 0.8304 - val_loss: 0.4892 - val_accuracy: 0.7669
Epoch 7/20
27/27 [==============================] - 1s 28ms/step - loss: 0.3625 - accuracy: 0.8328 - val_loss: 0.4866 - val_accuracy: 0.7859
Epoch 8/20
2

In [52]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.5440323948860168, 0.7615176439285278]

In [53]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.54
accuracy: 76.15%
